In [2]:
import cv2
import face_recognition

# Stream Camera Display w/OpenCV:

In [5]:
# Keystrokes to close display 
# window: q, esc, or <space bar>

KEY_Q     = 113
KEY_ESC   = 27
KEY_SPACE = 32

def cleanup(camera, win_name):
    camera.release()
    cv2.destroyWindow(win_name)
    cv2.waitKey(1)

In [ ]:
def display_camera(win_name='camera'):
#{
    camera = cv2.VideoCapture(0)
    cv2.namedWindow(win_name)
    cv2.startWindowThread()

    try:
    #{
        while True:
            vsuccess, cameraframe = camera.read()
            cv2.imshow(win_name, cameraframe)

            k = cv2.waitKey(30)
            if k == KEY_Q or k == KEY_ESC or k == KEY_SPACE:
                cleanup(camera, win_name)
                break
    #}  
    except KeyboardInterrupt as keyinterrupt:
        cleanup(camera, win_name)
        raise keyinterrupt # Rethrow to notebook cell
#}

display_camera()

# Face Detection Applied to Camera Stream:

In [3]:
# Experimented with including face profile detection with:
# profile_config = '../config/haarcascade_profileface.xml'
# profile_detector = cv2.CascadeClassifier(profile_config)

# But decided against it for now because the detector was 
# already causing camera lag, plus it only detects left side 
# profiles, and would thus require an image flip and a full 
# second pass through the detector to be fully effective.

def face_detect_camera(win_name='camera'):
#{    
    camera = cv2.VideoCapture(0)
    cv2.namedWindow(win_name)
    cv2.startWindowThread()

    frontal_config = '../config/haarcascade_frontalface_default.xml'
    frontal_detector = cv2.CascadeClassifier(frontal_config)

    try:
    #{
        while True:
        #{
            # Read camera frame
            vsuccess, cameraframe = camera.read()
            
            # Create gray scale version of frame and run classifiers
            greyframe = cv2.cvtColor(cameraframe, cv2.COLOR_BGR2GRAY)
            facerects = frontal_detector.detectMultiScale(greyframe, 1.1, 4)
                                    
            # Draw detected rectangles onto frame
            for (x, y, w, h) in facerects: cv2.rectangle(
                cameraframe, (x, y), (x+w, y+h), (255, 0, 0), 2)
            
            # Display
            cv2.imshow(win_name, cameraframe)
            k = cv2.waitKey(30)
            
            # Catch window close keystrokes
            if k == KEY_Q or k == KEY_ESC or k == KEY_SPACE:
                cleanup(camera, win_name)
                break
        #}
    #}
    except KeyboardInterrupt as keyinterrupt:
        cleanup(camera, win_name)
        raise keyinterrupt # Rethrow to notebook cell
#}

face_detect_camera()

In [6]:
def face_detect_camera2(win_name='camera'):
#{    
    camera = cv2.VideoCapture(0)
    cv2.namedWindow(win_name)
    cv2.startWindowThread()

    frontal_config = '../config/haarcascade_frontalface_default.xml'
    frontal_detector = cv2.CascadeClassifier(frontal_config)

    try:
    #{
        while True:
        #{
            # Read camera frame
            vsuccess, cameraframe = camera.read()
            
            # Run classifiers
            facerects = face_recognition.face_locations(cameraframe)
                                                
            # Draw detected rectangles onto frame
            for (top, rht, bot, lft) in facerects: cv2.rectangle(
                cameraframe, (lft, top), (rht, bot), (255, 0, 0), 2)
            
            # Display
            cv2.imshow(win_name, cameraframe)
            k = cv2.waitKey(30)
            
            # Catch window close keystrokes
            if k == KEY_Q or k == KEY_ESC or k == KEY_SPACE:
                cleanup(camera, win_name)
                break
        #}
    #}
    except KeyboardInterrupt as keyinterrupt:
        cleanup(camera, win_name)
        raise keyinterrupt # Rethrow to notebook cell
#}

face_detect_camera2()

# Face Detection Applied to Dataset
#### Using OpenCV Haar Cascade:

In [ ]:
def face_detect_dataset_cascade(istart=1, istop=None):
#{
    # Like range args
    if istop is None:
        istart, istop = 0, istart

    frontal_config = '../config/haarcascade_frontalface_default.xml'
    frontal_detector = cv2.CascadeClassifier(frontal_config)
        
    for i in range(istart, istop):
    #{
        try:
        #{
            if not os.path.isdir(df.frectdir(i)): os.mkdir(df.frectdir(i))
            for videoname in df.DataSplitter(i, 0.0, 0.25, False).train_split:
            #{
                datapath = f"{df.frectdir(i)}/{re.split(r'[/.]', videoname)[-2]}"
                if not os.path.isdir(datapath): os.mkdir(datapath)
                    
                _face_detect_video(f"{df.traindir(i)}/{videoname}", datapath, frontal_detector)
            #}
        #}
        except PermissionError as err: print("ERROR:", err)
    #}
#}

def _face_detect_video(videopath, datapath, fdetector):
#{
    video = cv2.VideoCapture(videopath)
    count, rcount, vsuccess = 0, 0, True
    
    initial = time.time()
    while video.isOpened() and vsuccess:
    #{
        vsuccess, videoframe = video.read()
        
        if vsuccess:
            # Create gray scale version of frame and run classifiers
            greyframe = cv2.cvtColor(videoframe, cv2.COLOR_BGR2GRAY)
            facerects = fdetector.detectMultiScale(greyframe, 1.1, 4)
                                    
            # Draw detected rectangles onto frame
            for (x, y, w, h) in facerects: cv2.rectangle(
                videoframe, (x, y), (x+w, y+h), (255, 0, 0), 2)
            
            if isinstance(facerects, np.ndarray):
                cv2.imwrite(f"{datapath}/frectframe{count}.jpg", videoframe)
                rcount += 1
                
        count += 1
    #}
    
    print(f"{videopath.split("/")[-1]} : numb face frames:{rcount}/{count-1} ({int(rcount/(count-1)*100)}%) in {time.time()-initial:.2f} sec")
#}

face_detect_dataset_cascade()

#### Using dlib face_detection: https://github.com/ageitgey/face_recognition

In [ ]:
def face_detect_dataset2(istart=1, istop=None):
#{
    # Like range args
    if istop is None:
        istart, istop = 0, istart
        
    for i in range(istart, istop):
    #{
        try:
        #{
            if not os.path.isdir(df.frectdir(i)): os.mkdir(df.frectdir(i))
            for videoname in df.DataSplitter(i, 0.0, 0.25, False).train_split:
            #{
                datapath = f"{df.frectdir(i)}/{re.split(r'[/.]', videoname)[-2]}"
                if not os.path.isdir(datapath): os.mkdir(datapath)
                    
                rcount = _face_detect_video2(f"{df.traindir(i)}/{videoname}", datapath, scale=0.25)
                if rcount < 0: _face_detect_video2(f"{df.traindir(i)}/{videoname}", datapath, scale=0.5)
                
            #}
        #}
        except PermissionError as err: print("ERROR:", err)
    #}
#}

def _face_detect_video2(videopath, datapath, scale=0.25):
#{
    video = cv2.VideoCapture(videopath)
    count, rcount, vsuccess = 0, 0, True
    revscale = 1.0/scale
    
    initial = time.time()
    while video.isOpened() and vsuccess:
    #{
        vsuccess, videoframe = video.read()
        
        if vsuccess:
            # Scale down by 1/4, convert from BGR to RGB, run face locator
            smallrgbframe = cv2.resize(videoframe, (0, 0), fx=scale, fy=scale)[:, :, ::-1]
            facerects = face_recognition.face_locations(smallrgbframe)
            
            # Draw detected (scaled back up) rectangles onto frame
            for (top, rht, bot, lft) in map(functools.partial(_revscale, rscale=revscale), facerects):
                cv2.rectangle(videoframe, (lft, top), (rht, bot), (255, 0, 0), 2)
                        
            if facerects:
                cv2.imwrite(f"{datapath}/frectframe{count}.jpg", videoframe)
                rcount += 1
                
        count += 1
    #}
    
    vidirname = re.split("[/\.]", videopath)[-2]
    print(f"{vidirname} : numb face frames:{rcount}/{count-1} in {time.time()-initial:.2f} sec")
    return rcount
#}

def _revscale(quad, rscale): return [int(val*rscale) for val in quad]

face_detect_dataset2()